In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from skimage.filters import gabor
from sklearn.feature_extraction import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
def preprocess_frame(frame, size=(224, 224)):
    frame_resized = cv2.resize(frame, size)
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    return frame_rgb


In [ ]:
def gabor_feature_extraction(image):
    filtered_images = []
    for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]:
        for frequency in [0.1, 0.3, 0.5]:
            real, imag = gabor(image, frequency=frequency, theta=theta)
            filtered_images.append(real)
    return np.stack(filtered_images, axis=-1)


In [ ]:
def orb_keypoints(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors


In [ ]:
def non_max_suppression(keypoints, descriptors):
    # You can implement a simple non-maximum suppression algorithm here
    # Retain only the keypoints with strong responses
    # Placeholder function
    return keypoints, descriptors


In [ ]:
def build_xception_model():
    base_model = Xception(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def deepfake_detection_pipeline(video_path):
    cap = cv2.VideoCapture(video_path)
    model = build_xception_model()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        preprocessed_frame = preprocess_frame(frame)
        gabor_features = gabor_feature_extraction(preprocessed_frame)
        keypoints, descriptors = orb_keypoints(preprocessed_frame)
        keypoints, descriptors = non_max_suppression(keypoints, descriptors)
        
        # Use CNN for classification
        prediction = model.predict(np.expand_dims(gabor_features, axis=0))
        print("Deepfake Detected" if prediction > 0.5 else "Authentic")

    cap.release()
